In [223]:
import textwrap

import pandas as pd

In [224]:
filepath = "xcku035ffva1156pkg.txt"

In [225]:
# Parse device name
with open(filepath, 'r') as f:
    device_name = f.readline().strip().split(sep=" ")[1]

# Parse the pinout from Xilinx ASCII Package File
pinout_table = pd.read_fwf(
    filepath, 
    colspecs=[(0, 6), (6, 42), (42, 61), (61, 67), (67, 77), (77, 97), (97, 107)],
    header=1,
    skipfooter=1,
    keep_default_na=False
    )

# pinout_table.sort_values(["Bank", "Pin"], inplace=True)

In [261]:
def create_symbol(device_name, pinout_table, banks):
    params = [
        '(property "Reference" "U" (at 0 0 0))',
        f'(property "Value" "{device_name.upper()}" (at 0 0 0))',
        '(property "Footprint" "" (at 0 0 0) (effects hide))',
        '(property "Datasheet" "" (at 0 0 0) (effects hide))',
        '(property "ki_keywords" "" (at 0 0 0) (effects hide))',
        '(property "ki_description" "" (at 0 0 0) (effects hide))',
        ]

    start = f"(symbol {device_name.upper()} (in_bom yes) (on_board yes)"
    bank_symbols = textwrap.indent("\n".join([create_bank(device_name, pinout_table, index, bank) for index, bank in enumerate(banks)]), "  ")
    end = ")"
    
    return "\n".join([start, textwrap.indent("\n".join(params), "  "), bank_symbols, end])
    return textwrap.dedent(symbol)


def create_bank(device_name, pinout_table, index, bank):
    start = f"(symbol {device_name.upper()}_{index+1}_1"
    pins  = textwrap.indent("\n".join([create_pin(pin) for _, pin in pinout_table.iterrows() if pin["Bank"] == bank]), "  ")
    end   = ")"
    
    return "\n".join([start, pins, end])


def create_pin(pin):
    power_names = ["VCC", "VDD", "VEE", "VBAT", "VP", "VN", "VSS", "GND"]
    io_types = ["CONFIG", "HR", "HP", "GTH"]
    
    if any(power_name in pin["Pin Name"] for power_name in power_names):
        pin_type = "power_in"
    elif any(io_type in pin["I/O Type"] for io_type in io_types):
        pin_type = "bidirectional"
    else:
        pin_type = "unspecified"

    symbol = f"(pin {pin_type} line (at 0 0 0) (length 2.54)\n(name {pin["Pin Name"]} (effects (font (size 1.27 1.27))))\n(number {pin["Pin"]} (effects (font (size 1.27 1.27)))))"
    return symbol

# Get unique values from Bank column
banks = list(set(pinout_table["Bank"]))

In [262]:
# print(create_symbol(device_name, pinout_table, banks))

with open("xilinx-auto-schematic.kicad_sym", "w") as f:
    f.write("(kicad_symbol_lib (version 20220914) (generator kicad_symbol_editor)\n")
    f.write(textwrap.indent(create_symbol(device_name, pinout_table, banks), "  "))
    f.write(")")

# print("\n".join([create_pin(pin) for _, pin in pinout_table.iterrows() if pin["Bank"] == banks[0]]))